# 保存模型 读取模型，做预测，使用签名函数

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras
print(tf.__version__)
print(sys.version_info)
for module in np, pd ,sklearn, tf, keras:
    print(module.__name__,module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:

fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
#x = (x - u) / std

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaler = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaler = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaler = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)


In [4]:
#tf.keras.models.Sequential()

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape =[28,28]),
    keras.layers.Dense(64,activation='relu'),
#     keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(10)#,activation='softmax')
])    

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer = 'sgd',
             metrics = ['accuracy'])


In [5]:
history = model.fit(x_train_scaler,y_train,epochs=10,
         validation_data=(x_valid_scaler,y_valid))


Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 12s 223us/sample - loss: 0.5718 - accuracy: 0.7982 - val_loss: 0.4398 - val_accuracy: 0.8488
Epoch 2/10
55000/55000 [==============================] - 11s 200us/sample - loss: 0.4255 - accuracy: 0.8488 - val_loss: 0.4016 - val_accuracy: 0.8620
Epoch 3/10
55000/55000 [==============================] - 12s 226us/sample - loss: 0.3897 - accuracy: 0.8614 - val_loss: 0.3863 - val_accuracy: 0.8654
Epoch 4/10
55000/55000 [==============================] - 11s 209us/sample - loss: 0.3684 - accuracy: 0.8698 - val_loss: 0.3804 - val_accuracy: 0.8646
Epoch 5/10
55000/55000 [==============================] - 10s 189us/sample - loss: 0.3520 - accuracy: 0.8739 - val_loss: 0.3618 - val_accuracy: 0.8746
Epoch 6/10
55000/55000 [==============================] - 10s 184us/sample - loss: 0.3385 - accuracy: 0.8803 - val_loss: 0.3476 - val_accuracy: 0.8790
Epoch 7/10
55000/55000 [=====================

In [6]:
model.evaluate(x_test_scaler,y_test,verbose = 2)

10000/1 - 1s - loss: 0.2276 - accuracy: 0.8636


[0.38180840286016465, 0.8636]

In [8]:
#保存模型,会在指定文件夹下生成文件saved_model.pb
tf.saved_model.save(model, 'data/keras_saved_graph')

INFO:tensorflow:Assets written to: data/keras_saved_graph/assets


In [14]:
# 查看saved_model信息，含有模型签名signature_def
# 前面加！在notebook中运行命令行工具
!saved_model_cli show --dir data/keras_saved_graph --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


In [16]:
# 使用命令行工具打印指定签名信息，打印signature_def  serving_default
!saved_model_cli show --dir data/keras_saved_graph \
                --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [21]:
# 对保存的model进行测试
!saved_model_cli run --dir data/keras_saved_graph \
                --tag_set serve \
                --signature_def serving_default\
                --input_exprs "flatten_input = np.ones((2,28,28))"

Traceback (most recent call last):
  File "/Users/mac/opt/anaconda3/envs/mytf/bin/saved_model_cli", line 11, in <module>
    sys.exit(main())
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/tools/saved_model_cli.py", line 643, in run
    init_tpu=args.init_tpu, tf_debug=args.tf_debug)
  File "/Users/mac/opt/anaconda3/envs/mytf/lib/python3.7/site-packages/tensorflow_core/python/tools/saved_model_cli.py", line 316, in run_saved_model_with_feed_dict
    (input_key_name, '"' + '", "'.join(inputs_tensor_info.keys()) + '"'))
ValueError: "flatten_input " is not a valid input key. Please choose from "flatten_input", or use --show option.


In [22]:
# 载入模型
loaded_model = tf.saved_model.load('data/keras_saved_graph')
# 打印所以签名
print(list(loaded_model.signatures.keys()))


['serving_default']


In [24]:
inference = loaded_model.signatures['serving_default']
print(inference)

In [27]:
print(inference.structured_outputs)

{'dense_1': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_1')}


In [29]:
results = inference(tf.constant(x_test_scaler[:1]))
print(results['dense_1'])

tf.Tensor(
[[-1.0564426 -1.988424  -1.2645077 -5.414941  -2.7713263  7.419647
  -1.1249225  7.0812345  4.2149386 10.055031 ]], shape=(1, 10), dtype=float32)
